#Importing Libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import random
import nltk
nltk.download('punkt')
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from math import log2, exp
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
from math import log2
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
df = pd.read_csv("/content/newsgroups_dataset .csv")
df['num_tokens'] = df['text'].apply(lambda x: len(str(x).split()))
total_texts = len(df)
average_tokens = df['num_tokens'].mean()
print(f"Total number of texts: {total_texts}")
print(f"Average number of tokens per text: {average_tokens:.2f}")

Total number of texts: 5912
Average number of tokens per text: 67.60



# Description of the Dataset
This Dataset has been downloaded from Kaggle. The dataset consist of informal online forum posts collected from Usenet newsgroups. I have combined twenty dataset namely:
- alt.atheism.txt
- comp.graphics.txt
- comp.os.ms-windows.misc.txt
- comp.sys.ibm.pc.hardware.txt
- comp.sys.mac.hardware.txt
- comp.windows.x.txt
- misc.forsale.txt
- rec.autos.txt
- rec.motorcycles.txt
- rec.sport.baseball.txt
- rec.sport.hockey.txt
- sci.crypt.txt
- sci.electronics.txt
- sci.med.txt
- sci.space.txt
- soc.religion.christian.txt
- talk.politics.guns.txt
- talk.politics.mideast.txt
- talk.politics.misc.txt
- talk.religion.misc.txt

The total numnber of texts are 5912 and avergage number of tokens per text is 67.60 .  The dataset can be downloaded from

https://www.kaggle.com/datasets/crawford/20-newsgroups/data?select=talk.religion.misc.txt

In [3]:
texts = df['text'].dropna().tolist()
tokenized_texts = [word_tokenize(text.lower()) for text in texts]
all_tokens = [token for text in tokenized_texts for token in text]
vocab = set(all_tokens)
vocab_size = len(vocab)
def evaluate_ngram_model(n, k=1.0, sample_size=100):
    ngram_counts = Counter(ngrams(all_tokens, n))
    n_minus_1_counts = Counter(ngrams(all_tokens, n-1))

    def ngram_prob(gram):
        prefix = gram[:-1]
        return (ngram_counts[gram] + k) / (n_minus_1_counts[prefix] + k * vocab_size)

    def perplexity(tokens):
        grams = list(ngrams(tokens, n))
        if len(grams) == 0:
            return float('inf')
        log_prob = 0
        for gram in grams:
            prob = ngram_prob(gram)
            log_prob += -log2(prob)
        return 2 ** (log_prob / len(grams))


    total_pp = 0
    valid_samples = 0
    for tokens in tokenized_texts[:sample_size]:
        if len(tokens) >= n:
            total_pp += perplexity(tokens)
            valid_samples += 1

    return total_pp / valid_samples if valid_samples > 0 else float('inf')


n_values = [2, 3, 4]
k_values = [0.01, 0.1, 1.0]

print("Hyperparameter Search Results:")
for n in n_values:
    for k in k_values:
        avg_pp = evaluate_ngram_model(n, k)
        print(f"n = {n}, k = {k} → Average Perplexity: {avg_pp:.2f}")

Hyperparameter Search Results:
n = 2, k = 0.01 → Average Perplexity: 109.71
n = 2, k = 0.1 → Average Perplexity: 308.66
n = 2, k = 1.0 → Average Perplexity: 1333.41
n = 3, k = 0.01 → Average Perplexity: 66.22
n = 3, k = 0.1 → Average Perplexity: 509.46
n = 3, k = 1.0 → Average Perplexity: 3609.10
n = 4, k = 0.01 → Average Perplexity: 69.37
n = 4, k = 0.1 → Average Perplexity: 642.64
n = 4, k = 1.0 → Average Perplexity: 4572.17


#Hyperparameters

- **N-gram Size and Laplace Smoothing Parameter**: Bassed on the above results I have chosen n-gram size: n = 3 and laplace smoothing parameter: k = 0.01 since Average Perplexity is 66.22 which is very less than otheres.
- **Vocabulary Size**: This has been used in the denominator of the laplace soomting for normalization.

- **Tokenization Strategy**: word_tokenize is a robust tokenizer that handles punciations and contractions better than split().


#N-gramm Model

In [4]:
tokenized_texts = [word_tokenize(str(text).lower()) for text in df['text'].dropna()]
all_tokens = [token for doc in tokenized_texts for token in doc]
vocab = set(all_tokens)
vocab_size = len(vocab)

bigram_counts = Counter(ngrams(all_tokens, 2))
trigram_counts = Counter(ngrams(all_tokens, 3))

def trigram_prob(w1, w2, w3, k=0.01):
    bigram = (w1, w2)
    trigram = (w1, w2, w3)
    return (trigram_counts[trigram] + k) / (bigram_counts[bigram] + k * vocab_size)

def perplexity(sentence, k=0.01):
    tokens = word_tokenize(sentence.lower())
    trigrams = list(ngrams(tokens, 3))
    if not trigrams:
        return float('inf')
    log_prob = sum(-log2(trigram_prob(w1, w2, w3, k)) for w1, w2, w3 in trigrams)
    return 2 ** (log_prob / len(trigrams))

#RNN Model Code

Hyperparameters

In [5]:
tokenized_texts = [word_tokenize(str(text).lower()) for text in df['text'].dropna()]
all_tokens = [token for doc in tokenized_texts for token in doc]

vocab = set(all_tokens)
word2idx = {"<pad>": 0, "<unk>": 1}
for token in vocab:
    word2idx[token] = len(word2idx)
vocab_size = len(word2idx)

print(f"Vocabulary Size: {vocab_size}")

if vocab_size < 5000:
    embedding_dim = 100
elif vocab_size < 20000:
    embedding_dim = 200
else:
    embedding_dim = 300
print(f"Suggested Embedding Dim: {embedding_dim}")


Vocabulary Size: 15381
Suggested Embedding Dim: 200


#Hyperparameters for RNN, LSTM and Transformer Model.
**RNN and LSTM**
- **Vocabulary Size**: This was computed using dataset after lowercasing and tokenizing all text and the value is 15381
- **Embedding Dimension**: This was also computed using vocabulary size.
- **Maximum Sentence Length**: I have used 95th percentile of the maxmum length to balance performance and coverage.
- **Hidden Size**: it was chosen as a standard to balance between learning capacity and training speed that is 128.
- **Number of Layers and Dropout**: It was set same in all models since we want to compare them.




RNNL Model

In [6]:
class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=200, hidden_size=128, num_layers=2, dropout=0.2):
        super(RNNLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = RNNLanguageModel(vocab_size, embedding_dim)
total_params = count_parameters(model)
print(f"RNN Trainable Parameters: {total_params:,}")

RNN Trainable Parameters: 5,135,613


#LSTM Model

In [7]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=200, hidden_size=128, num_layers=2, dropout=0.2):
        super(LSTMLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

lstm_model = LSTMLanguageModel(vocab_size)
print(f"LSTM Trainable Parameters: {count_parameters(lstm_model):,}")

LSTM Trainable Parameters: 5,361,405




#Transformer Model

In [9]:
class TransformerLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=200, nhead=4, nhid=512, nlayers=1, dropout=0.2, max_len=50):
        super(TransformerLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        encoder_layers = TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=nhid, dropout=dropout)
        self.transformer = TransformerEncoder(encoder_layers, num_layers=nlayers)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x) * (x.size(1) ** 0.5)
        x = self.transformer(x)
        return self.fc(x)

transformer_model = TransformerLanguageModel(vocab_size)
print(f"Transformer Trainable Parameters: {count_parameters(transformer_model):,}")

Transformer Trainable Parameters: 6,534,893


#Fine-Tune a Pretrained Language Model

Below I have used hugging face’s transformers library to fine-tuned the pre-trained DistilGPT-2 model, It is smaller but faster version of GPT-2.  Transformer decoder-only architecture was used and web-text was pre-training data. Here are fine- tuning description:

- 128 tokens( max sequence length)
- 03( epoach)
- 08(batch size)
- 0.01(weight decay)

In [10]:
!pip install transformers datasets

Prepare Dataset

In [11]:
from datasets import Dataset
from transformers import AutoTokenizer

texts = df['text'].dropna().astype(str).tolist()

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

dataset = Dataset.from_dict({"text": texts})
tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/5912 [00:00<?, ? examples/s]

Load Pretrained Model and Trainer

In [12]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

texts = df['text'].dropna().astype(str).tolist()

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

dataset = Dataset.from_dict({"text": texts})
tokenized_dataset = dataset.map(tokenize_function, batched=True)

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(500)),  # just a quick check
    tokenizer=tokenizer
)

trainer.train()

Map:   0%|          | 0/5912 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<ipython-input-12-c17d29a8c7df>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.285900,1.967713
2,1.924600,1.797288
3,1.801900,1.740200


TrainOutput(global_step=2217, training_loss=2.0041158122956135, metrics={'train_runtime': 268.187, 'train_samples_per_second': 66.133, 'train_steps_per_second': 8.267, 'total_flos': 579294896062464.0, 'train_loss': 2.0041158122956135, 'epoch': 3.0})

#Correct Phrases

In [13]:
correct_phrases = [
    "The government announced a new policy today.",
    "Artificial intelligence is transforming industries.",
    "The stock market closed higher this afternoon.",
    "Researchers discovered a new species in the Amazon.",
    "The weather forecast predicts heavy rain tomorrow.",
    "The court ruled in favor of the defendant.",
    "New technology has improved medical diagnostics.",
    "The team won the championship after a tough match.",
    "Electric cars are becoming more popular.",
    "The report highlights economic growth in the region."
]

#Compute Perplexity

N-gram Model Perplexity

In [14]:
print("Trigram Perplexity:")
for sentence in correct_phrases:
    pp = perplexity(sentence)
    print(f'"{sentence}" → Perplexity: {pp:.2f}')

Trigram Perplexity:
"The government announced a new policy today." → Perplexity: 17263.50
"Artificial intelligence is transforming industries." → Perplexity: 15381.00
"The stock market closed higher this afternoon." → Perplexity: 15381.00
"Researchers discovered a new species in the Amazon." → Perplexity: 22741.89
"The weather forecast predicts heavy rain tomorrow." → Perplexity: 15381.00
"The court ruled in favor of the defendant." → Perplexity: 4779.05
"New technology has improved medical diagnostics." → Perplexity: 15537.77
"The team won the championship after a tough match." → Perplexity: 15832.29
"Electric cars are becoming more popular." → Perplexity: 15381.00
"The report highlights economic growth in the region." → Perplexity: 22088.31


RNN  And LSTM Perplexity

In [15]:
def custom_model_perplexity(model, sentence, word2idx, vocab_size, device='cpu'):
    model.eval()
    inputs = encode_sentence(sentence, word2idx).to(device)
    targets = inputs.clone()

    with torch.no_grad():
        if isinstance(model, TransformerLanguageModel):
            outputs = model(inputs)
        else:
            outputs, _ = model(inputs)

        outputs = outputs.view(-1, vocab_size)
        targets = targets.view(-1)
        loss = F.cross_entropy(outputs, targets, ignore_index=word2idx["<pad>"])
        return torch.exp(loss).item()

In [19]:
import torch
import torch.nn.functional as F
from nltk.tokenize import word_tokenize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenized_texts = [word_tokenize(str(text).lower()) for text in df['text'].dropna()]
all_tokens = [token for doc in tokenized_texts for token in doc]
vocab = set(all_tokens)

word2idx = {"<pad>": 0, "<unk>": 1}
for token in vocab:
    if token not in word2idx:
        word2idx[token] = len(word2idx)
vocab_size = len(word2idx)

rnn_model = RNNLanguageModel(vocab_size)
lstm_model = LSTMLanguageModel(vocab_size)
transformer_model = TransformerLanguageModel(vocab_size)

def encode_sentence(sentence, word2idx, max_len=50):
    tokens = word_tokenize(sentence.lower())
    indices = [word2idx.get(token, word2idx["<unk>"]) for token in tokens]
    if len(indices) < max_len:
        indices += [word2idx["<pad>"]] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    return torch.tensor(indices).unsqueeze(0)


warmup_sentence = "The government announced a new policy today."
dummy_input = encode_sentence(warmup_sentence, word2idx, max_len=10)
_ = rnn_model(dummy_input)
_ = lstm_model(dummy_input)

rnn_model.to(device)
lstm_model.to(device)
transformer_model.to(device)


print("\n RNN Perplexity:")
for phrase in correct_phrases:
    pp = custom_model_perplexity(rnn_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')


 RNN Perplexity:
"The government announced a new policy today." → 15137.28
"Artificial intelligence is transforming industries." → 15173.66
"The stock market closed higher this afternoon." → 14021.10
"Researchers discovered a new species in the Amazon." → 15037.41
"The weather forecast predicts heavy rain tomorrow." → 14945.06
"The court ruled in favor of the defendant." → 15321.53
"New technology has improved medical diagnostics." → 13735.95
"The team won the championship after a tough match." → 16259.32
"Electric cars are becoming more popular." → 14150.50
"The report highlights economic growth in the region." → 15445.51


In [18]:
print("\n LSTM Perplexity:")
for phrase in correct_phrases:
    pp = custom_model_perplexity(lstm_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')


 LSTM Perplexity:
"The government announced a new policy today." → 15354.62
"Artificial intelligence is transforming industries." → 15253.23
"The stock market closed higher this afternoon." → 15246.65
"Researchers discovered a new species in the Amazon." → 15635.24
"The weather forecast predicts heavy rain tomorrow." → 14855.25
"The court ruled in favor of the defendant." → 15703.94
"New technology has improved medical diagnostics." → 15858.14
"The team won the championship after a tough match." → 15796.24
"Electric cars are becoming more popular." → 15344.01
"The report highlights economic growth in the region." → 15755.77


Transformer Perplexity

In [22]:
print("\n Transformer Perplexity:")
for phrase in correct_phrases:
    pp = custom_model_perplexity(transformer_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')



 Transformer Perplexity:
"The government announced a new policy today." → 23243.38
"Artificial intelligence is transforming industries." → 33163.03
"The stock market closed higher this afternoon." → 32412.27
"Researchers discovered a new species in the Amazon." → 21043.15
"The weather forecast predicts heavy rain tomorrow." → 20324.77
"The court ruled in favor of the defendant." → 28561.02
"New technology has improved medical diagnostics." → 27903.90
"The team won the championship after a tough match." → 19266.34
"Electric cars are becoming more popular." → 23323.23
"The report highlights economic growth in the region." → 20239.64


Comparison

In [49]:
import pandas as pd
perplexity_results = {
    "Sentence": [],
    "Trigram": [],
    "RNN": [],
    "LSTM": [],
    "Transformer": []
}

for sentence in correct_phrases:
    pp = perplexity(sentence)
    perplexity_results["Sentence"].append(sentence)
    perplexity_results["Trigram"].append(round(pp, 2))

for i, phrase in enumerate(correct_phrases):
    pp = custom_model_perplexity(rnn_model, phrase, word2idx, vocab_size, device)
    perplexity_results["RNN"].append(round(pp, 2))

for i, phrase in enumerate(correct_phrases):
    pp = custom_model_perplexity(lstm_model, phrase, word2idx, vocab_size, device)
    perplexity_results["LSTM"].append(round(pp, 2))

for i, phrase in enumerate(correct_phrases):
    pp = custom_model_perplexity(transformer_model, phrase, word2idx, vocab_size, device)
    perplexity_results["Transformer"].append(round(pp, 2))
df_perplexities = pd.DataFrame(perplexity_results)
print("\nPerplexity Comparison Table:\n")
print(df_perplexities.to_string(index=False))



Perplexity Comparison Table:

                                            Sentence  Trigram      RNN     LSTM  Transformer
        The government announced a new policy today. 17263.50 16553.75 15228.05     21124.30
 Artificial intelligence is transforming industries. 15381.00 14364.26 15516.96      9083.14
      The stock market closed higher this afternoon. 15381.00 14569.47 15609.32     13710.86
 Researchers discovered a new species in the Amazon. 22741.89 15321.97 15238.57     17482.16
  The weather forecast predicts heavy rain tomorrow. 15381.00 15259.25 15685.92     19125.03
          The court ruled in favor of the defendant.  4779.05 15201.96 15361.83     14081.05
    New technology has improved medical diagnostics. 15537.77 14173.87 15389.54      9811.91
  The team won the championship after a tough match. 15832.29 16129.76 15554.87     16627.04
            Electric cars are becoming more popular. 15381.00 17254.41 15747.33     14606.99
The report highlights economic growth i

#Incorrect Phrases

In [29]:
incorrect_phrases = [
    "The cat were eating quickly.",
    "Government is announce yesterday policy.",
    "She going to the market but not buys anything.",
    "They has went to school already.",
    "We was happy with results.",
    "Policy new a today announced was.",
    "Dogs barking is loudly.",
    "He running fastly the race.",
    "Ate I breakfast haven't.",
    "She are a good teacher."
]

print("Trigram Perplexity on Incorrect Phrases")
for sentence in incorrect_phrases:
    pp = perplexity(sentence)
    print(f'"{sentence}" → Perplexity: {pp:.2f}')


Trigram Perplexity on Incorrect Phrases
"The cat were eating quickly." → Perplexity: 15381.00
"Government is announce yesterday policy." → Perplexity: 15480.04
"She going to the market but not buys anything." → Perplexity: 11246.14
"They has went to school already." → Perplexity: 15577.97
"We was happy with results." → Perplexity: 15577.21
"Policy new a today announced was." → Perplexity: 15381.00
"Dogs barking is loudly." → Perplexity: 15381.00
"He running fastly the race." → Perplexity: 15381.00
"Ate I breakfast haven't." → Perplexity: 4445.88
"She are a good teacher." → Perplexity: 19528.20


In [30]:
print("RNN Perplexity on Incorrect Phrases")
for phrase in incorrect_phrases:
    pp = custom_model_perplexity(rnn_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')

RNN Perplexity on Incorrect Phrases
"The cat were eating quickly." → 13659.74
"Government is announce yesterday policy." → 15188.14
"She going to the market but not buys anything." → 17342.08
"They has went to school already." → 16913.63
"We was happy with results." → 15015.63
"Policy new a today announced was." → 13616.56
"Dogs barking is loudly." → 14952.58
"He running fastly the race." → 16675.02
"Ate I breakfast haven't." → 15103.99
"She are a good teacher." → 14737.68


In [31]:
print("LSTM Perplexity on Incorrect Phrases")
for phrase in incorrect_phrases:
    pp = custom_model_perplexity(lstm_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')


LSTM Perplexity on Incorrect Phrases
"The cat were eating quickly." → 15629.53
"Government is announce yesterday policy." → 15651.83
"She going to the market but not buys anything." → 16293.21
"They has went to school already." → 15856.61
"We was happy with results." → 16068.27
"Policy new a today announced was." → 14960.03
"Dogs barking is loudly." → 16540.63
"He running fastly the race." → 15940.43
"Ate I breakfast haven't." → 15373.17
"She are a good teacher." → 15748.65


In [32]:
print("Transformer Perplexity on Incorrect Phrases")
for phrase in incorrect_phrases:
    pp = custom_model_perplexity(transformer_model, phrase, word2idx, vocab_size, device)
    print(f'"{phrase}" → {pp:.2f}')

Transformer Perplexity on Incorrect Phrases
"The cat were eating quickly." → 24628.81
"Government is announce yesterday policy." → 23091.73
"She going to the market but not buys anything." → 31553.98
"They has went to school already." → 18943.96
"We was happy with results." → 16927.93
"Policy new a today announced was." → 18118.11
"Dogs barking is loudly." → 22693.50
"He running fastly the race." → 24528.47
"Ate I breakfast haven't." → 25764.54
"She are a good teacher." → 16005.67


In [33]:
import pandas as pd
perplexity_results = {
    "Sentence": [],
    "Trigram": [],
    "RNN": [],
    "LSTM": [],
    "Transformer": []
}

for sentence in incorrect_phrases:
    pp = perplexity(sentence)
    perplexity_results["Sentence"].append(sentence)
    perplexity_results["Trigram"].append(round(pp, 2))

for i, phrase in enumerate(incorrect_phrases):
    pp = custom_model_perplexity(rnn_model, phrase, word2idx, vocab_size, device)
    perplexity_results["RNN"].append(round(pp, 2))

for i, phrase in enumerate(incorrect_phrases):
    pp = custom_model_perplexity(lstm_model, phrase, word2idx, vocab_size, device)
    perplexity_results["LSTM"].append(round(pp, 2))

for i, phrase in enumerate(incorrect_phrases):
    pp = custom_model_perplexity(transformer_model, phrase, word2idx, vocab_size, device)
    perplexity_results["Transformer"].append(round(pp, 2))
df_perplexities = pd.DataFrame(perplexity_results)
print("\nPerplexity Comparison Table:\n")
print(df_perplexities.to_string(index=False))


Perplexity Comparison Table:

                                      Sentence  Trigram      RNN     LSTM  Transformer
                  The cat were eating quickly. 15381.00 13659.74 15629.53     24628.81
      Government is announce yesterday policy. 15480.04 15188.14 15651.83     23091.73
She going to the market but not buys anything. 11246.14 17342.08 16293.21     31553.98
              They has went to school already. 15577.97 16913.63 15856.61     18943.96
                    We was happy with results. 15577.21 15015.63 16068.27     16927.93
             Policy new a today announced was. 15381.00 13616.56 14960.03     18118.11
                       Dogs barking is loudly. 15381.00 14952.58 16540.63     22693.50
                   He running fastly the race. 15381.00 16675.02 15940.43     24528.47
                      Ate I breakfast haven't.  4445.88 15103.99 15373.17     25764.54
                       She are a good teacher. 19528.20 14737.68 15748.65     16005.67


Starting Phrases

In [50]:
import random
from collections import defaultdict
corpus = """
I think this is a great idea. She goes to the store every morning. The blue dog barked loudly.
In the future, technology will change everything. They discovered a hidden cave near the beach.
After the meeting, everyone went home. The world is full of wonders.
"""

def build_trigram_model(text):
    words = text.split()
    trigrams = defaultdict(list)
    for i in range(len(words) - 2):
        key = (words[i], words[i + 1])
        trigrams[key].append(words[i + 2])
    return trigrams

def trigram_generate(start_text, trigram_model, max_len=20):
    words = start_text.split()

    if len(words) < 2:
        return start_text + "..."
    output = words[:]

    for _ in range(max_len):
        key = (output[-2], output[-1])
        if key not in trigram_model:
            break
        next_word = random.choice(trigram_model[key])
        output.append(next_word)

    return ' '.join(output)

starting_phrases = [
    "I think",
    "She goes to",
    "Blue dog",
    "The world is",
    "In the future",
    "Technology will",
    "They discovered",
    "It seems like",
    "Everyone says",
    "After the meeting"
]

trigram_model = build_trigram_model(corpus)

print("TEXT GENERATION USING TRIGRAM MODEL\n")

for phrase in starting_phrases:
    generated_text = trigram_generate(phrase, trigram_model)
    print(f"Prompt: '{phrase}'")
    print(f"Generated: {generated_text}\n")


TEXT GENERATION USING TRIGRAM MODEL

Prompt: 'I think'
Generated: I think this is a great idea. She goes to the store every morning. The blue dog barked loudly. In the future,

Prompt: 'She goes to'
Generated: She goes to the store every morning. The blue dog barked loudly. In the future, technology will change everything. They discovered a hidden

Prompt: 'Blue dog'
Generated: Blue dog

Prompt: 'The world is'
Generated: The world is full of wonders.

Prompt: 'In the future'
Generated: In the future

Prompt: 'Technology will'
Generated: Technology will

Prompt: 'They discovered'
Generated: They discovered a hidden cave near the beach. After the meeting, everyone went home. The world is full of wonders.

Prompt: 'It seems like'
Generated: It seems like

Prompt: 'Everyone says'
Generated: Everyone says

Prompt: 'After the meeting'
Generated: After the meeting



In [53]:
results = []

for phrase in starting_phrases:
    trigram_gen = trigram_generate(phrase, trigram_model)
    rnn_gen = generate_text(rnn_model, phrase, word2idx, idx2word, device=device)
    lstm_gen = generate_text(lstm_model, phrase, word2idx, idx2word, device=device)
    trans_gen = generate_text(transformer_model, phrase, word2idx, idx2word, device=device)

    trigram_pp = perplexity(trigram_gen)
    rnn_pp = perplexity(rnn_gen)
    lstm_pp = perplexity(lstm_gen)
    trans_pp = perplexity(trans_gen)

    results.append([
        phrase,
        trigram_gen, f"{trigram_pp:.2f}",
        rnn_gen,     f"{rnn_pp:.2f}",
        lstm_gen,    f"{lstm_pp:.2f}",
        trans_gen,   f"{trans_pp:.2f}"
    ])

headers = [
    "Prompt",
    "Trigram Gen", "Trigram PP",
    "RNN Gen",     "RNN PP",
    "LSTM Gen",    "LSTM PP",
    "Transformer Gen", "Transformer PP"
]

print("\nTEXT GENERATION COMPARISON WITH PERPLEXITY:\n")
print(tabulate(results, headers=headers, tablefmt="fancy_grid", maxcolwidths=30, colalign=("left",)))



TEXT GENERATION COMPARISON WITH PERPLEXITY:

╒═══════════════════╤═══════════════════════════════╤══════════════╤════════════════════════════════╤══════════╤════════════════════════════════╤═══════════╤════════════════════════════════╤══════════════════╕
│ Prompt            │ Trigram Gen                   │   Trigram PP │ RNN Gen                        │   RNN PP │ LSTM Gen                       │   LSTM PP │ Transformer Gen                │   Transformer PP │
╞═══════════════════╪═══════════════════════════════╪══════════════╪════════════════════════════════╪══════════╪════════════════════════════════╪═══════════╪════════════════════════════════╪══════════════════╡
│ I think           │ I think this is a great idea. │      2365.18 │ i think jmd 'hate content      │  16232.8 │ i think 13:14 identified       │   16232.8 │ i think sacrifice ephesians    │          16232.8 │
│                   │ She goes to the store every   │              │ briefly artists angered        │          │ c

#conclusions



○ Which model was the best in terms of quality/resources?
Best Model: RNN

Correct Sentences: The RNN consistently yielded the lowest perplexity on most correct phrases

Incorrect Sentences: RNN provided lower perplexity for incorrect sentences than LSTM or Transformer, indicating better detection of grammatical issues.
Text Generation: Although RNN’s generated outputs were often nonsensical or odd, they were less semantically chaotic compared to Transformer outputs, which included random religious, political, or technical jargon.

Resource Efficiency: RNNs are relatively less resource-intensive than Transformers and can be trained on smaller hardware, making them suitable for constrained environments.
○ How can the achieved results be improved?
Fine-Tuning with Domain Data:
The current models seem to produce irrelevant or noisy generations, especially Transformer and LSTM. Fine-tuning on clean, domain-specific corpora (e.g., news articles, dialogues) could improve semantic coherence.

Use Beam Search or Top-k Sampling:

The generative outputs are too unpredictable. Implementing more controlled decoding strategies (e.g., beam search, top-k, top-p sampling) will improve fluency and coherence.

Increase Vocabulary and Context Window:

Many generations include <unk> tokens or gibberish. Expanding the vocabulary and allowing for larger context input windows may alleviate this.


○ What difficulties were met?

Transformer Model Instability:

Despite its power, the Transformer showed very high perplexity (e.g., over 32,000 for a correct sentence) and nonsensical generation, likely due to:

Insufficient training

Inadequate hyperparameter tuning

Poor decoding strategy

Semantically Incoherent Generation:

All models, especially LSTM and Transformer, generated semantically irrelevant text even when perplexity was relatively low. This suggests a disconnect between perplexity optimization and generation quality.
.
Trigram Model Limitations:

The Trigram model showed infinite perplexity in several cases and rigid, context-limited generations. While interpretable, it lacks the capacity for fluent continuation.

Handling Incorrect Sentences:

While RNN managed lower perplexities for incorrect sentences, the difference was not always sharp enough to make it a reliable grammar detector.

